In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
cars = pd.read_csv("../data/raw/Cars.csv")
customers = pd.read_csv("../data/raw/Customers.csv")
sales = pd.read_csv("../data/raw/Sales.csv")

In [3]:
# cars.head()
# customers.head()
# sales.head()

cars.info()
customers.info()
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Car_ID             500 non-null    object
 1   Brand              500 non-null    object
 2   Model              500 non-null    object
 3   Year               500 non-null    int64 
 4   Color              500 non-null    object
 5   Engine_Type        500 non-null    object
 6   Transmission       500 non-null    object
 7    Price             500 non-null    object
 8   Quantity_In_Stock  500 non-null    int64 
 9   Status             500 non-null    object
dtypes: int64(2), object(8)
memory usage: 39.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  2000 non-null   object
 1   Name         2000 non-null   object
 2   Gende

In [4]:
def clean_columns(df):
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )
    return df

cars = clean_columns(cars)
customers = clean_columns(customers)
sales = clean_columns(sales)

In [5]:
# cars.columns
customers.columns
# sales.columns

Index(['customer_id', 'name', 'gender', 'age', 'phone', 'email', 'city'], dtype='object')

In [6]:
def clean_money(col):
    return (
        col.astype(str)
           .str.replace("$", "", regex=False)
           .str.replace(",", "", regex=False)
           .str.strip()
    )

cars["price"] = pd.to_numeric(clean_money(cars["price"]), errors="coerce")
sales["sale_price"] = pd.to_numeric(clean_money(sales["sale_price"]), errors="coerce")

In [7]:
# cars["price"].isna().sum()
sales["sale_price"].isna().sum()

0

In [8]:
sales["sale_date"] = pd.to_datetime(sales["sale_date"], dayfirst=True)
sales["sale_date"].isna().sum()

0

In [9]:
def clean_phone(phone):
    phone = str(phone)
    phone = phone.replace(" ", "").replace("-", "")
    
    if phone.startswith("+91"):
        phone = phone[3:]
    elif phone.startswith("91") and len(phone) > 10:
        phone = phone[2:]
        
    return phone

In [10]:
import re

def clean_phone(phone):
    phone = str(phone).lower()
    
    # Remove extension part
    phone = phone.split("x")[0]
    
    # Keep only digits
    phone = re.sub(r"\D", "", phone)
    
    # Keep last 10 digits (standard mobile format)
    if len(phone) > 10:
        phone = phone[-10:]
        
    return phone

In [11]:
customers["clean_phone"] = customers["phone"].apply(clean_phone)
customers[["phone", "clean_phone"]].head(10)

,phone,clean_phone
0,(527)170-6357,5271706357
1,324-959-9856x281,3249599856
2,461.428.6407,4614286407
3,119-340-1448x571,1193401448
4,514-684-2974x378,5146842974
5,(356)146-5556x78074,3561465556
6,(272)146-6686x12407,2721466686
7,061.225.2031,0612252031
8,558.988.9694x89389,5589889694
9,874-636-8430,8746368430


In [12]:
customers["clean_phone"].str.len().value_counts()

clean_phone
10    1891
4       79
9       19
5       10
3        1
Name: count, dtype: int64

In [13]:
customers["clean_phone"] = customers["clean_phone"].astype(str)
customers.loc[customers["clean_phone"].str.len() != 10, "clean_phone"] = np.nan

In [14]:
# customers["clean_phone"].isna().sum()
customers["clean_phone"].str.len().value_counts()

clean_phone
10.0    1891
Name: count, dtype: int64

In [15]:
cars.columns = cars.columns.str.strip()
customers.columns = customers.columns.str.strip()
sales.columns = sales.columns.str.strip()

In [16]:
print(cars.columns)
print(sales.columns)

Index(['car_id', 'brand', 'model', 'year', 'color', 'engine_type',
       'transmission', 'price', 'quantity_in_stock', 'status'],
      dtype='object')
Index(['sale_id', 'customer_id', 'car_id', 'sale_date', 'quantity',
       'sale_price', 'payment_method', 'salesperson'],
      dtype='object')


In [17]:
for col in cars.columns:
    print(repr(col))

'car_id'
'brand'
'model'
'year'
'color'
'engine_type'
'transmission'
'price'
'quantity_in_stock'
'status'


In [18]:
def clean_money(x):
    return float(str(x).replace("$", "").replace(",", "").strip())

cars["price"] = cars["price"].apply(clean_money)
sales["sale_price"] = sales["sale_price"].apply(clean_money)

In [19]:
cars["price"].dtype
sales["sale_price"].dtype

cars["price"].head()
sales["sale_price"].head()

0    73293.0
1    32681.0
2    53531.0
3    89817.0
4    77591.0
Name: sale_price, dtype: float64

In [20]:
cars["price"].dtype
sales["sale_price"].dtype

dtype('float64')

In [21]:
sales["sale_date"] = pd.to_datetime(sales["sale_date"], dayfirst=True, errors="coerce")

In [22]:
sales["sale_date"].dtype

dtype('<M8[ns]')

In [23]:
sales["total_amount"] = sales["sale_price"] * sales["quantity"]
sales[["sale_price", "quantity", "total_amount"]].head()

,sale_price,quantity,total_amount
0,73293.0,3,219879.0
1,32681.0,3,98043.0
2,53531.0,2,107062.0
3,89817.0,1,89817.0
4,77591.0,2,155182.0


In [24]:
sales_cars = sales.merge(cars, on="car_id", how="left")
master_df = sales_cars.merge(customers, on="customer_id", how="left")

In [25]:
master_df.shape
master_df.head()

,sale_id,customer_id,car_id,sale_date,quantity,sale_price,payment_method,salesperson,total_amount,brand,...,price,quantity_in_stock,status,name,gender,age,phone,email,city,clean_phone
0,S00001,CU1241,C0214,2025-03-28,3,73293.0,Installment,Ashley Ramos,219879.0,Toyota,...,64961.0,6,Sold,Valerie Harris,Female,70,828-990-2303x830,owenslisa@gmail.com,Doughertychester,8289902303
1,S00002,CU0100,C0202,2024-02-12,3,32681.0,Cash,Pamela Blair,98043.0,Mercedes,...,35183.0,8,Sold,Morgan Bowman,Female,42,001-052-982-0299,bbond@yahoo.com,Williamsonbury,0529820299
2,S00003,CU1690,C0228,2023-02-26,2,53531.0,Credit,Sergio Lee,107062.0,Toyota,...,80597.0,0,Sold,Marc Meza,Female,61,9996253975,swatkins@gmail.com,West David,9996253975
3,S00004,CU0534,C0231,2024-06-21,1,89817.0,Cash,Mary Johnston,89817.0,Toyota,...,41319.0,17,Available,Deborah Wilcox,Female,44,520.705.9024x429,rvasquez@cook.info,Stephanieville,5207059024
4,S00005,CU1153,C0071,2023-12-21,2,77591.0,Installment,Ricardo Garcia,155182.0,Kia,...,53636.0,4,Available,Jacqueline Wood,Female,26,-1736,roy31@gmail.com,Lake Ryanville,NaN


In [26]:
total_revenue = master_df["total_amount"].sum()
total_cars_sold = master_df["quantity"].sum()
total_customers = master_df["customer_id"].nunique()
total_transactions = master_df.shape[0]

total_revenue, total_cars_sold, total_customers, total_transactions

(1146524618.0, 19979, 1986, 10000)

In [27]:
master_df.groupby("brand")["total_amount"].sum().sort_values(ascending=False).head(10)

brand
Tesla       182178640.0
Mercedes    179989319.0
Nissan      175742300.0
BMW         166634399.0
Kia         157375178.0
Toyota      154467440.0
Hyundai     130137342.0
Name: total_amount, dtype: float64

In [28]:
master_df["month"] = master_df["sale_date"].dt.to_period("M")
master_df.groupby("month")["total_amount"].sum().head()

month
2022-10      370037.0
2022-11    35019171.0
2022-12    34237240.0
2023-01    31312060.0
2023-02    28617484.0
Freq: M, Name: total_amount, dtype: float64

In [29]:
master_df["year"] = master_df["sale_date"].dt.year
master_df["month_num"] = master_df["sale_date"].dt.month
master_df["month_name"] = master_df["sale_date"].dt.strftime("%B")
master_df["weekday"] = master_df["sale_date"].dt.day_name()

In [30]:
customer_ltv = master_df.groupby("customer_id").agg(
    total_spent=("total_amount", "sum"),
    total_orders=("sale_id", "count"),
    cars_bought=("quantity", "sum"),
    avg_order_value=("total_amount", "mean")
).reset_index()

customer_ltv.head()

,customer_id,total_spent,total_orders,cars_bought,avg_order_value
0,CU0001,332424.0,3,5,110808.000000
1,CU0002,626629.0,3,7,208876.333333
2,CU0003,415834.0,3,6,138611.333333
3,CU0004,460823.0,4,7,115205.750000
4,CU0005,497434.0,6,11,82905.666667


In [31]:
latest_date = master_df["sale_date"].max()

rfm = master_df.groupby("customer_id").agg(
    recency=("sale_date", lambda x: (latest_date - x.max()).days),
    frequency=("sale_id", "count"),
    monetary=("total_amount", "sum")
).reset_index()

rfm.head()

,customer_id,recency,frequency,monetary
0,CU0001,15,3,332424.0
1,CU0002,89,3,626629.0
2,CU0003,135,3,415834.0
3,CU0004,318,4,460823.0
4,CU0005,35,6,497434.0


In [32]:
master_df.groupby(["brand","model"])["quantity"].sum().sort_values(ascending=False).head(10)

brand     model  
Nissan    Sunny      1326
Tesla     Model X    1296
Toyota    Camry      1266
Mercedes  GLA        1173
Nissan    Qashqai    1093
Hyundai   Tucson     1085
Mercedes  E-Class    1006
Kia       Cerato      971
          Seltos      969
Tesla     Model S     966
Name: quantity, dtype: int64

In [33]:
master_df.groupby(["brand","model"])["total_amount"].sum().sort_values(ascending=False).head(10)

brand     model   
Toyota    Camry       73576440.0
Nissan    Sunny       73414184.0
Tesla     Model X     73390873.0
Mercedes  GLA         66568138.0
Nissan    Qashqai     63807125.0
Hyundai   Tucson      61560206.0
Mercedes  E-Class     59556123.0
BMW       5 Series    56422102.0
Kia       Cerato      55991593.0
BMW       3 Series    55576958.0
Name: total_amount, dtype: float64

In [34]:
cars.sort_values("quantity_in_stock").head(10)

,car_id,brand,model,year,color,engine_type,transmission,price,quantity_in_stock,status
98,C0099,Hyundai,Tucson,2018,Gray,Diesel,Automatic,38984.0,0,Reserved
393,C0394,BMW,X5,2016,Black,Diesel,Automatic,38266.0,0,Reserved
427,C0428,Mercedes,E-Class,2016,Black,Hybrid,Manual,46983.0,0,Reserved
356,C0357,Tesla,Model X,2019,Gray,Hybrid,Automatic,55936.0,0,Sold
29,C0030,Hyundai,Tucson,2022,Gray,Diesel,Manual,30893.0,0,Available
87,C0088,Kia,Cerato,2024,White,Hybrid,Automatic,92417.0,0,Sold
454,C0455,Nissan,Qashqai,2023,Gray,Hybrid,Manual,94666.0,0,Sold
44,C0045,BMW,5 Series,2020,White,Electric,Manual,85822.0,0,Reserved
305,C0306,Nissan,Qashqai,2021,Blue,Diesel,Manual,80288.0,0,Sold
463,C0464,Hyundai,Elantra,2023,White,Hybrid,Manual,71865.0,0,Sold


In [35]:
monthly_trend = master_df.groupby(["year","month_num"])["total_amount"].sum().reset_index()
monthly_trend.head()

,year,month_num,total_amount
0,2022,10,370037.0
1,2022,11,35019171.0
2,2022,12,34237240.0
3,2023,1,31312060.0
4,2023,2,28617484.0


In [36]:
weekday_sales = master_df.groupby("weekday")["total_amount"].sum().reset_index()
weekday_sales

,weekday,total_amount
0,Friday,161105571.0
1,Monday,160287109.0
2,Saturday,166957945.0
3,Sunday,168183413.0
4,Thursday,162904936.0
5,Tuesday,167427634.0
6,Wednesday,159658010.0


In [37]:
fact_sales = master_df[[
    "sale_id","sale_date","customer_id","car_id",
    "quantity","sale_price","total_amount","salesperson"
]]

dim_customers = customers.copy()
dim_cars = cars.copy()
dim_date = master_df[["sale_date","year","month_num","month_name","weekday"]].drop_duplicates()

In [38]:
fact_sales.to_csv("fact_sales.csv", index=False)
dim_customers.to_csv("dim_customers.csv", index=False)
dim_cars.to_csv("dim_cars.csv", index=False)
dim_date.to_csv("dim_date.csv", index=False)